In [ ]:
import keras
import cv2
import numpy as np
from glob import glob

# 相対パスを書くために、現在のディレクトリの位置を書き出す
import os
os.getcwd()


In [37]:
def data_load(path, img_width, img_height, CLS, Framework='Tensorflow'):
    """ファイルパスからデータセットをロードするための関数"""
    xs = [] # 訓練データ用の空リスト
    ts = [] # データラベル用の空リスト
    paths = [] # ロードしたファイルパス用の空リスト
    img_read_err = [] # ロードできなかったファイルパス用の空リスト
    num_classes = len(CLS)+1

    # データをロードするためにディレクトリを下りていく
    for dir_path_1 in glob(DirPath):
        for dir_path_2 in glob(dir_path_1 + '/*'):
            for path in glob(dir_path_2 + '/*'):
                #print(path, 'を読み込みました。')

                # 訓練用画像を読み込む
                x = cv2.imread(path)
                if x is None: # もし，画像がロードできなかった場合
                    print(path, 'を読み込めませんでした．')
                    img_read_err.append(path)
                    continue
                else: # 画像がロードできた場合
                    x = cv2.resize(x, (img_width, img_height)).astype(np.float32)
                    x /= 255.
                    xs.append(x)
                
                # 正解ラベルを作成する
                lbl = dir_path_1[dir_path_1.find('\\'):].strip('\\')
                # 使用するフレームワークによって，正解ラベルの作成方法が異なる
                if (Framework is 'PyTorch'):
                    t = float(lbl)
                elif (Framework is 'Tensorflow') or (Framework is 'Keras'):
                    # one-hot-labelを作成する
                    t = np.zeros(num_classes)
                    for i, cls in enumerate(CLS):
                        if cls == lbl:
                            t[i] = 1

                ts.append(t)
                paths.append(path)

    xs = np.array(xs, dtype=np.float32)
    ts = np.array(ts, dtype=np.int)

    return xs, ts, paths, img_read_err

In [39]:
# LeNetを作成する
## Network
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization

def LeNet(img_width, img_height, num_classes):
    inputs = Input((img_height, img_width, 3))
    x = Conv2D(6, (5, 5), padding='valid', activation=None, name='conv1')(inputs)
    x = MaxPool2D((2, 2), padding='same')(x)
    x = Activation('sigmoid')(x)
    x = Conv2D(16, (5, 5), padding='valid', activation=None, name='conv2')(x)
    x = MaxPool2D((2, 2), padding='same')(x)
    x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(120, name='dense1', activation=None)(x)
    x = Dense(64, name='dense2', activation=None)(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x, name='model')
    return model

In [45]:
def train(DirPath, img_size, cls_label):
    model = LeNet(img_size[0], img_size[1], len(cls_label)+1)

    for layer in model.layers:
        layer.trainable = True

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
        metrics=['accuracy'])
    
    xs, ts, paths, _ = data_load(DirPath, img_size[0], img_size[1], cls_label)

    # training
    mb = 100
    mbi = 0
    count = 1
    loss_ls = []

    loss_ave = 0
    loss_AveMin = 1
    loss_AveCnt = 0
    #loss_AveLs = []

    train_ind = np.arange(len(xs))
    np.random.seed(0)
    np.random.shuffle(train_ind)

    #for i in range(500):
    while True:
        if mbi + mb > len(xs):
            mb_ind = train_ind[mbi:]
            np.random.shuffle(train_ind)
            mb_ind = np.hstack((mb_ind, train_ind[:(mb-(len(xs)-mbi))]))
            mbi = mb - (len(xs) - mbi)
            
            loss_ave = np.average(loss_ls)
            if loss_AveMin > loss_ave:
                loss_AveMin = loss_ave
                loss_AveCnt = 0
            else: loss_AveCnt += 1
            loss_ls = []
            # print('iter >>', count, ',loss_ave >>', loss_ave, 'accuracy >>', acc)
        else:
            mb_ind = train_ind[mbi: mbi+mb]
            mbi += mb

        x = xs[mb_ind]
        t = ts[mb_ind]

        loss, acc = model.train_on_batch(x=x, y=t)
        print('iter >>', count, ',loss >>', loss, 'accuracy >>', acc)
        
        loss_ls.append(loss)
        count += 1

        if loss_AveCnt == 50: break

    model.save('LeNet.h5')
    return 0

In [44]:
# データを読み込むフォルダを指定する
DirPath = '../../../DataSet/AngleDetection/training/*'
#print(DirPath)

num_classes = 36
img_width, img_height = 64, 64
CLS = np.arange(0, 175, 5).astype('str')

#train(DirPath, img_size=(64, 64), cls_label=CLS)
train(DirPath, img_size=(img_width, img_height), cls_label=CLS)

 ,loss >> 2.2524884 accuracy >> 0.35
iter >> 362 ,loss >> 1.8011628 accuracy >> 0.36
iter >> 363 ,loss >> 1.9804626 accuracy >> 0.34
iter >> 364 ,loss >> 2.2828422 accuracy >> 0.32
iter >> 365 ,loss >> 2.0984051 accuracy >> 0.38
iter >> 366 ,loss >> 1.8410748 accuracy >> 0.32
iter >> 367 ,loss >> 2.0234895 accuracy >> 0.36
iter >> 368 ,loss >> 1.9331087 accuracy >> 0.37
iter >> 369 ,loss >> 2.8728411 accuracy >> 0.31
iter >> 370 ,loss >> 2.5228205 accuracy >> 0.28
iter >> 371 ,loss >> 1.9322217 accuracy >> 0.34
iter >> 372 ,loss >> 2.7281756 accuracy >> 0.31
iter >> 373 ,loss >> 2.3368287 accuracy >> 0.43
iter >> 374 ,loss >> 2.181413 accuracy >> 0.31
iter >> 375 ,loss >> 2.860215 accuracy >> 0.29
iter >> 376 ,loss >> 2.2072654 accuracy >> 0.44
iter >> 377 ,loss >> 2.280387 accuracy >> 0.42
iter >> 378 ,loss >> 1.9181503 accuracy >> 0.3
iter >> 379 ,loss >> 2.183417 accuracy >> 0.31
iter >> 380 ,loss >> 1.6529671 accuracy >> 0.38
iter >> 381 ,loss >> 1.9956174 accuracy >> 0.4
iter >> 3

KeyboardInterrupt: 

In [38]:
DirPath = '../../../DataSet/AngleDetection/training/*'
img_size = (64, 64)
CLS = np.arange(0, 175, 5).astype('str')

xs, ts, paths, _ = data_load(DirPath, img_size[0], img_size[1], CLS)

In [40]:
print(xs.shape)
print(ts.shape)

mb = 100
mbi = 0

train_ind = np.arange(len(xs))
np.random.seed(0)
np.random.shuffle(train_ind)

mb_ind = train_ind[mbi: mbi+mb]
mbi += mb

x = xs[mb_ind]
t = ts[mb_ind]

print(x.shape)
print(t.shape)

(30694, 64, 64, 3)
(30694, 36)
(100, 64, 64, 3)
(100, 36)
